<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [440]:
import pandas as pd
import psycopg2

In [443]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [444]:
# текст запроса
query_3_1 = f'''
SELECT 
    COUNT(id) as total_vacancies
FROM public.vacancies 
'''

In [445]:
# результат запроса
df_vacancies = pd.read_sql_query(query_3_1, connection)
df_vacancies

,total_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [446]:
# текст запроса
query_3_2 = f'''SELECT  
    COUNT(id) as employers_total
from public.employers
'''

In [447]:
# результат запроса
df_employers = pd.read_sql_query(query_3_2, connection)
df_employers

,employers_total
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [448]:
# текст запроса
query_3_3 = f'''
SELECT 
    COUNT(id) as areas_total
FROM public.areas
'''

In [449]:
# результат запроса
df_areas = pd.read_sql_query(query_3_3, connection)
df_areas

,areas_total
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [450]:
# текст запроса
query_3_4 = f''' 
SELECT
    COUNT(id) as industries_total
FROM public.industries 
'''

In [451]:
# результат запроса
df_industries = pd.read_sql_query(query_3_4, connection)
df_industries

,industries_total
0,294


In [452]:
print(f'Количество опубликованных вакансий: \n {df_vacancies.loc[:1]} \n',
      f'\n Количество работодателей: \n {df_employers.loc[:1]} \n',
      f'\n Количество регионов: \n {df_areas.loc[:1]} \n',
      f'\n Количество сфер деятельности: \n {df_industries.loc[:1]}'
      )

Количество опубликованных вакансий: 
    total_vacancies
0            49197 
 
 Количество работодателей: 
    employers_total
0            23501 
 
 Количество регионов: 
    areas_total
0         1362 
 
 Количество сфер деятельности: 
    industries_total
0               294


***

#### Выводы по предварительному анализу
По предварительному анализу данных можно сделать вывод, что на каждого работодателя в среднем публикуется более двух вакансий.
На каждую сферу деятельности в среднем приходится 80 работодателей и 167 вакансий.
Предварительный анализ даёт очень поверхностное представление, для полноты картины и более серьёзных выводов необходимо провести детальный анализ в разрезе представленных данных.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [453]:
# текст запроса
query_4_1 = f''' 
SELECT 
    a.name area,
    COUNT(v.id) cnt
FROM public.areas a
JOIN public.vacancies v ON a.id = v.area_id
GROUP BY 1
ORDER BY 2 desc
'''

In [454]:
# результат запроса
df_vacancies_by_area = pd.read_sql_query(query_4_1, connection)
df_vacancies_by_area

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [455]:
# текст запроса
query_4_2 = f''' 
SELECT
    COUNT(id) 
FROM public.vacancies
WHERE salary_to is not NULL or salary_from is not NULL 

'''

In [456]:
# результат запроса
df_salary = pd.read_sql_query(query_4_2, connection)
df_salary

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [457]:
# текст запроса
query_4_3 = ''' 
SELECT
    ROUND(AVG(salary_from), 0) "нижняя граница",
    ROUND(AVG(salary_to), 0) "верхняя граница"
FROM public.vacancies
'''

In [458]:
# результат запроса
df_average_salary = pd.read_sql_query(query_4_3, connection)
df_average_salary

,нижняя граница,верхняя граница
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [459]:
# текст запроса
query_4_4 = ''' 
SELECT
    schedule "график", 
    employment "тип занятости",
    COUNT(schedule)
FROM public.vacancies
GROUP BY 1, 2
ORDER BY 3 desc
'''

In [460]:
# результат запроса
df_schedule = pd.read_sql_query(query_4_4, connection)
df_schedule

,график,тип занятости,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [461]:
# текст запроса
query_4_5 = ''' 
SELECT
    experience "опыт работы",
    COUNT(id)
FROM public.vacancies
GROUP BY 1
ORDER BY 2
'''

In [462]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)
df_experience

,опыт работы,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [463]:
print(f'ТОП-3 городов по количеству вакансий: \n {df_vacancies_by_area.iloc[:3]} \n',
      f'\n Зарплата указана у \n {df_employers.loc[:1]} вакансий \n',
      f'\n Средняя зарплатная вилка лежит в диапазоне: \n {df_average_salary.iloc[:1]} рублей \n',
      f'\n ТОП-3 востребованных графиков работы: \n {df_schedule.iloc[:3]} \n',
      f'\n Распределение опыта работы в вакансиях: \n {df_experience}'
      )

ТОП-3 городов по количеству вакансий: 
               area   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112 
 
 Зарплата указана у 
    employers_total
0            23501 вакансий 
 
 Средняя зарплатная вилка лежит в диапазоне: 
    нижняя граница  верхняя граница
0         71065.0         110537.0 рублей 
 
 ТОП-3 востребованных графиков работы: 
              график     тип занятости  count
0       Полный день  Полная занятость  35367
1  Удаленная работа  Полная занятость   7802
2     Гибкий график  Полная занятость   1593 
 
 Распределение опыта работы в вакансиях: 
           опыт работы  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


#### Выводы по детальному анализу вакансий
1. Средняя зарплатная вилка, вероятно, не отражает ситуации в регионах, поскольку 20% вакансий приходится на крупные города - Москва, Санкт-Петербург и Минск, стоит отметить, что лишь в 50% вакансий указана зарплата.
2. Самой востребованной является полная занятость, при этом полный рабочий день установлен в 70% вакансиях, удалённая работа на втором месте по популярности, это, вероятно, связано с адаптацией работодателей к условиям пандемии.
3. Количество вакансий находится в обратной зависимости от увеличения опыта работы. Это может быть связано с тем, что опытных специалистов чаще выращивают внутри компании или нанимают по рекомендациям, а кандидатов на освободившиеся вакансии ищут на внешних ресурсах.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [516]:
# текст запроса
query_5_1 = ''' 
(SELECT
    e.name,
    COUNT(v.id)
FROM public.vacancies v
JOIN public.employers e ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 desc  
LIMIT 1)
    UNION
(SELECT
    e.name,
    COUNT(v.id)
FROM public.vacancies v
JOIN public.employers e ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 desc  
LIMIT 1 OFFSET 4)
ORDER BY 2 desc
'''

In [517]:
# результат запроса
df_employers = pd.read_sql_query(query_5_1, connection)
df_employers

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [503]:
# текст запроса
query_5_2 = ''' 
SELECT
    a.name area,
    COUNT(DISTINCT e.name) employers,
    COUNT(DISTINCT v.name) vacancies
FROM public.areas a 
LEFT JOIN public.employers e on a.id = e.area
LEFT JOIN public.vacancies v on v.area_id = a.id
WHERE v.name is NULL
GROUP BY 1
ORDER BY 2 desc
'''

In [504]:
# результат запроса
df_vacancies_by_area = pd.read_sql_query(query_5_2, connection)
df_vacancies_by_area

,area,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [468]:
# текст запроса
query_5_3 = ''' 
SELECT
    e.name employer,
    COUNT(DISTINCT a.name)
FROM public.vacancies v
LEFT JOIN public.employers e ON v.employer_id = e.id
LEFT JOIN public.areas a ON v.area_id = a.id
GROUP BY 1
ORDER BY 2 desc
'''

In [469]:
# результат запроса
df_employers_areas = pd.read_sql_query(query_5_3, connection)
df_employers_areas

,employer,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [470]:
# текст запроса
query_5_4 = ''' 
SELECT
    COUNT(e.name),
    ei.industry_id
FROM employers e
LEFT JOIN employers_industries ei on ei.employer_id = e.id
WHERE ei.industry_id is NULL
GROUP BY 2
'''


In [471]:
# результат запроса
df_industry_null = pd.read_sql_query(query_5_4, connection)
df_industry_null

,count,industry_id
0,8419,None


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [472]:
# текст запроса
query_5_5 = ''' 
SELECT
    e.name employer,
    COUNT(ei.industry_id)
FROM public.employers_industries ei
JOIN public.employers e ON e.id = ei.employer_id
GROUP BY 1
HAVING COUNT(ei.industry_id) = 4
ORDER BY 1
LIMIT 1 OFFSET 2

'''

In [473]:
# результат запроса
df_four_industries = pd.read_sql_query(query_5_5, connection)
df_four_industries

,employer,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [509]:
# текст запроса
query_5_6 = ''' 
SELECT 
    i.name industries,
    COUNT(ei.employer_id) employer
FROM employers_industries ei
JOIN industries i ON ei.industry_id = i.id
WHERE i.name LIKE '%Разработка программного обеспечения%'
GROUP BY 1

'''

In [510]:
# результат запроса
df_dev_industries = pd.read_sql_query(query_5_6, connection)
df_dev_industries

,industries,employer
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.


In [476]:
# код для получения списка городов-милионников
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population' # Определяем адрес страницы
cities = pd.read_html(url)[0] # Извлекаем таблицу с городами
#
cities = tuple(cities[cities[('Population', '2021 Census[1]')]>1000000].iloc[:, 2]) # фильтруем по городам-миллионникам
display(cities)


('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [477]:
# текст запроса
query_5_7 = f''' 
SELECT 
    a.name,
    COUNT(v.name)
    from vacancies v 
    JOIN areas a ON a.id = v.area_id
    JOIN employers e ON v.employer_id = e.id
    WHERE a.name IN {cities} and e.name = 'Яндекс'
    GROUP BY 1
        UNION
SELECT 
    'total',
    COUNT(v.id)
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
JOIN areas a ON a.id = v.area_id
WHERE a.name IN {cities} and e.name = 'Яндекс'
ORDER BY 2
'''

In [478]:
# результат запроса
df_yandex_vacancies = pd.read_sql_query(query_5_7, connection)
df_yandex_vacancies

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

#### Выводы по анализу работодателей
1. Яндекс является крупнейшим работодателем и имеет 1933 вакансий в 181 регионе, из них 485 вакансий - в городах-миллионниках, 
больше всего в Москве - 54 вакансии. 
2. Помимо Яндекс в ТОП-5 крупнейших работодателей входят Ростелеком, Тинькофф, СБЕР и Газпромнефть.
2. Самыми востребованными являются вакансии в сфере IT.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [480]:
# текст запроса
query_6_1 = ''' 
SELECT
    COUNT(name) data_vacancies
FROM vacancies
WHERE name ILIKE '%data%' or name ILIKE '%данн%' 
'''

In [481]:
# результат запроса
df_data_vacancies = pd.read_sql_query(query_6_1, connection)
df_data_vacancies

,data_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [524]:
# текст запроса
query_6_2 = ''' 
SELECT
    name
FROM vacancies
WHERE (name ilike '%data science%' 
    OR name ilike '%data scientist%' 
    OR name ilike '%исследователь данных%' 
    OR (name like '%ML%' and name not like '%HTML%') 
    OR name ilike '%machine learning%' 
    OR name ilike '%машинн%обучен%')
    and (employment ilike '%Стажировка%' 
    OR experience ilike '%Нет%опыта%' 
    OR name ilike '%junior%') 
'''

In [525]:
# результат запроса
df_ds_vacancies = pd.read_sql_query(query_6_2, connection)
df_ds_vacancies

,name
0,Junior+ CV/ML developer
1,Data Scientist (Junior+)
2,Junior Data Scientist (Стажер)
3,"Математик, Junior/Middle/Senior Data Scientist"
4,Junior Data Scientist
5,Data Scientist/Machine Learning Engineer (Deve...
6,Data Scientist (Junior)
7,Intern/Junior Data Scientist
8,Junior Data Scientist (VK)
9,Junior/Middle Data Scientist


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [484]:
# текст запроса
query_6_3 = ''' 
SELECT
    COUNT(key_skills)
FROM vacancies
WHERE (name ilike '%data science%' 
    OR name ilike '%data scientist%' 
    OR name ilike '%исследователь данных%' 
    OR (name like '%ML%' and name not like '%HTML%') 
    OR name ilike '%machine learning%' 
    OR name ilike '%машинн%обучен%')
    and (key_skills ilike '%SQL%' OR key_skills ilike '%postgres%')
    '''

In [485]:
# результат запроса
df_ds_sql = pd.read_sql_query(query_6_3, connection)
df_ds_sql

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [486]:
# текст запроса
query_6_4 = ''' 
SELECT
    COUNT(key_skills)
FROM vacancies
WHERE (name ilike '%data science%' 
    OR name ilike '%data scientist%' 
    OR name ilike '%исследователь данных%' 
    OR (name like '%ML%' and name not like '%HTML%') 
    OR name ilike '%machine learning%' 
    OR name ilike '%машинн%обучен%')
    and key_skills ilike '%Python%'
    '''

In [487]:
# результат запроса
df_ds_python = pd.read_sql_query(query_6_4, connection)
df_ds_python

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [488]:
# текст запроса
query_6_5 = ''' 
SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9),'')) + 1), 2) avg_skills
FROM vacancies
WHERE (name ilike '%data science%' 
    OR name ilike '%data scientist%' 
    OR name ilike '%исследователь данных%' 
    OR (name like '%ML%' and name not like '%HTML%') 
    OR name ilike '%machine learning%' 
    OR name ilike '%машинн%обучен%')
    and key_skills is NOT NULL 
'''

In [489]:
# результат запроса
df_avg_skills = pd.read_sql_query(query_6_5, connection)
df_avg_skills

,avg_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [490]:
# текст запроса
query_6_6 = ''' 
SELECT 
    experience, 
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary
FROM vacancies
WHERE  (name ilike '%data science%' 
    OR name ilike '%data scientist%' 
    OR name ilike '%исследователь данных%' 
    OR (name like '%ML%' and name not like '%HTML%') 
    OR name ilike '%machine learning%' 
    OR name ilike '%машинн%обучен%')
    AND experience is NOT NULL
    
    GROUP BY 1
'''

In [491]:
# результат запроса
df_ds_salary = pd.read_sql_query(query_6_6, connection)
df_ds_salary

,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

#### Выводы по предметному анализу
1. С работой с данными связана 1771 вакансия, что составляет 3% от общего числа вакансий.
2. В сфере, связанной с Data Science, размещено 480 вакансий, из них 51 вакансия для соискателей без опыта работы.
3. В 201 вакансии (42%) в DS в качестве ключевого навыка указан SQL/Postgres, Python в качестве ключевого навыка указан в 351 вакансии (73%).
4. В среднем для вакансий в области DS работодатели указывают больше 6 ключевых навыков.
5. Средняя зарплата для специалистов DS начинается от 74.6 тыс. рублей и доходит до 243 т.р. в зависимости от опыта работы, что в целом выше, чем средняя зарплата по всему рынку (средняя верхняя граница для всех вакансий - 110 т.р.).
При этом стоит отметить, что для специалистов с опытом работы более 6 лет зарплата не указана, вероятно, это связано с тем, что для опытных специалистов вопрос оплаты труда обсуждается индивидуально и зависит от многих факторов.


## Дополнительные задания и общие выводы по проекту

Оценим распределение средней зарплаты по регионам для сферы DS

In [639]:
query_add_1 = ''' 
SELECT 
    a.name,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(v.name) vacancies
FROM vacancies v
JOIN areas a ON v.area_id = a.id
WHERE  v.name ilike '%data science%' 
    OR v.name ilike '%data scientist%' 
    OR v.name ilike '%исследователь данных%' 
    OR (v.name like '%ML%' and v.name not like '%HTML%')
    OR v.name ilike '%machine learning%' 
    OR v.name ilike '%машинн%обучен%'
GROUP BY 1
HAVING COUNT(v.name) > 5
ORDER BY 2 desc
'''

In [640]:
query_salary_ds_region = pd.read_sql_query(query_add_1, connection)
query_salary_ds_region

,name,round,count
0,Нижний Новгород,NaN,20
1,Томск,NaN,8
2,Нур-Султан,NaN,6
3,Краснодар,NaN,6
4,Минск,NaN,9
5,Армения,268863.0,6
6,Москва,207630.0,217
7,Санкт-Петербург,173335.0,64
8,Новосибирск,166333.0,23
9,Екатеринбург,124000.0,7


Рассмотрим распределение средних зарплат в DS в разрезе типа рабочего графика

In [659]:
query_add_2 = ''' 
SELECT 
    v.schedule,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(v.name) vacancies
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE 
    v.name ilike '%data science%' 
    OR v.name ilike '%data scientist%' 
    OR v.name ilike '%исследователь данных%' 
    OR (v.name like '%ML%' and v.name not like '%HTML%')
    OR v.name ilike '%machine learning%' 
    OR v.name ilike '%машинн%обучен%'
GROUP BY 1
HAVING COUNT(v.name) > 5
ORDER BY 2 desc
LIMIT 5
'''

In [660]:
query_salary_schedule = pd.read_sql_query(query_add_2, connection)
query_salary_schedule

,schedule,round,count
0,Удаленная работа,190043.0,115
1,Полный день,175726.0,322
2,Гибкий график,140500.0,43


Рассмотрим поподробнее вакансии с удалённой работой, какой тип занятости и опыт работы востребованы в этих вакансиях

In [679]:
query_add_3_1 = ''' 
SELECT 
    v.experience,
    v.employment,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(v.name) vacancies
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE v.schedule = 'Удаленная работа'
AND (v.name ilike '%data science%' 
    OR v.name ilike '%data scientist%' 
    OR v.name ilike '%исследователь данных%' 
    OR (v.name like '%ML%' and v.name not like '%HTML%')
    OR v.name ilike '%machine learning%' 
    OR v.name ilike '%машинн%обучен%')
GROUP BY 1, 2
ORDER BY 3

'''

In [680]:
query_top_employers = pd.read_sql_query(query_add_3_1, connection)
query_top_employers

,experience,employment,round,count
0,От 1 года до 3 лет,Проектная работа,90000.0,2
1,От 3 до 6 лет,Частичная занятость,119000.0,2
2,От 1 года до 3 лет,Полная занятость,179224.0,38
3,От 3 до 6 лет,Полная занятость,232048.0,42
4,Нет опыта,Полная занятость,NaN,3
5,Более 6 лет,Полная занятость,NaN,27
6,От 1 года до 3 лет,Частичная занятость,NaN,1


In [692]:
query_add_3_2 = ''' 
SELECT 
    a.name area,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(DISTINCT e.name) employers
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
JOIN areas a ON e.area = a.id
WHERE v.schedule = 'Удаленная работа'
AND (v.name ilike '%data science%' 
    OR v.name ilike '%data scientist%' 
    OR v.name ilike '%исследователь данных%' 
    OR (v.name like '%ML%' and v.name not like '%HTML%')
    OR v.name ilike '%machine learning%' 
    OR v.name ilike '%машинн%обучен%')
GROUP BY 1
HAVING COUNT(v.name) > 5
ORDER BY 3 desc
'''

In [693]:
df_dist_work_by_region = pd.read_sql_query(query_add_3_2, connection)
df_dist_work_by_region

,area,avg_salary,employers
0,Москва,211429.0,30
1,Россия,NaN,3
2,Санкт-Петербург,163837.0,3
3,Екатеринбург,NaN,1


In [ ]:
query_add_3_1 = ''' 
SELECT 
    v.experience,
    v.employment,
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
    COUNT(v.name) vacancies
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE v.schedule = 'Удаленная работа'

GROUP BY 1, 2
ORDER BY 3

'''

In [759]:
query_add_4 = ''' 
(SELECT
i.name,
ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) avg_salary,
COUNT(v.name) vacancies
FROM industries i
JOIN employers_industries ie ON i.id = ie.industry_id
JOIN employers e ON ie.employer_id = e.id
JOIN vacancies v ON e.id = v.employer_id
where v.key_skills ilike '%Python%' 
GROUP BY 1
HAVING COUNT(v.name) > 5 and ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) is not NULL
)
    UNION 
(SELECT 
    'total', 
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))),
    COUNT(v.name)
    FROM vacancies v
    where v.key_skills ilike '%Python%' 
    )
    ORDER BY 3
'''

In [761]:
df_salaries_py = pd.read_sql_query(query_add_4, connection)
df_salaries_py


,name,avg_salary,vacancies
0,Розничная сеть (спортивные товары),60000.0,4
1,"Продукция авиационной, аэрокосмической, вертол...",117100.0,5
2,Инфраструктурная компания в энергетике (СО ЕЭС...,41600.0,5
3,"Нефтехимия (продвижение, оптовая торговля)",50000.0,5
4,"Земледелие, растениеводство, животноводство",22500.0,5
...,...,...,...
84,Банк,109938.0,231
85,"Интернет-компания (поисковики, платежные систе...",154234.0,383
86,"Системная интеграция, автоматизации технологи...",152590.0,532
87,Разработка программного обеспечения,179630.0,760


#### Итоги исследования (дополнительные материалы)
1. В рамках проекта был рассмотрен рынок труда по регионам, сферам деятельности и по количеству вакансий в компаниях, выявлены зависимости среднего уровня зарплат от типа занятости, опыта работы и географии регионов.
2. Проведён детальный анализ вакансий, размещённых в сфере DS, сравнительный анализ уровня зарплат на всём рынке с зарплатами в области IT.
3. Зарплата в IT выше, чем в среднем по рынку, кроме того наблюдается высокое расслоение уровня зарплат в разрезе регионов.
4. Стоит также отметить, что удаленный график работы занимает второе место по полулярности и, вероятно, тенденция к росту этой доли будет увеличиваться в условиях цифровизации бизнеса.

Выводы по дополнительным материалам:
1. По количеству вакансий в DS лидирует Москва, Санкт-Петербург и Новосибирск. Самые высокие средние зарплаты в Армении, Москве и Санк-Петербурге (рассматривались только регионы, в которых размещено более 5 вакансий)
2. 115 вакансий в области DS (24%) предлагают удалённый график работы, при этом уровень заработных плат не уступает вакансиям, предусматривающим работу в офисе, но стоит отметить, что для специалистов без опыта работы удалённых вакансий практически нет. Большинство вакансий с удалённой работой зарегистрированы в Москве (30 компаний) и подавляющее большинство рассматривают только полный рабочий день.
3. Всего вакансий, где в ключевых навыках указан Python - 3461 в 89 отраслях (принимались в расчёт только те, в которых размещено более 5 вакансий), средняя зарплата составляет 143 т.р., что соответсвует среднему уровню зарплат в сфере DS.

